In [3]:
import numpy as np 
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt

import cufflinks as cf
cf.set_config_file(offline = True)

import yfinance as yf

# Ignore warnings - optional
import warnings
warnings.filterwarnings('ignore')

C:\Users\rahul\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [5]:
# Calculating option prices through simulation
nsteps = 252
nsims = 10000
S0, K, rf, T, vol = 100, 105, 0.05, 1, 0.3

dt = 1 / nsteps

# Generate random paths for the stock price using GBM
def stock_price_paths(S0, rf, T, vol, nsteps, nsims):
    S = np.zeros((nsims, nsteps + 1))
    S[:, 0] = S0
    for i in range(nsteps):
        phi = np.random.randn(nsims)
        S[:, i + 1] = S[:, i] * np.exp((rf - 0.5 * vol**2) * dt + vol * phi * np.sqrt(dt))
    return S

paths = stock_price_paths(S0, rf, T, vol, nsteps, nsims)


option_prices = np.zeros(nsteps)
for t in range(1, nsteps + 1):
    option_payoffs = np.maximum(paths[:,:t].max(axis=1) - K, 0)
    option_prices[t - 1] = np.exp(-rf * t* dt) *option_payoffs.mean()


option_price_df = pd.DataFrame({'Option price': option_prices})
option_price_df

,Option price
0,0.000000
1,0.002558
2,0.038132
3,0.115306
4,0.224166
...,...
247,22.213422
248,22.276786
249,22.344881
250,22.406966


In [6]:
stock_price = yf.download('GOOG', start = '2022-11-14' ,end = '2023-11-15',progress = False, interval = '1d')
stock_price.index = stock_price.index.date

In [7]:
Prices = stock_price['Adj Close']
Prices

2022-11-14     96.029999
2022-11-15     98.720001
2022-11-16     98.989998
2022-11-17     98.500000
2022-11-18     97.800003
                 ...    
2023-11-08    133.259995
2023-11-09    131.690002
2023-11-10    134.059998
2023-11-13    133.639999
2023-11-14    135.429993
Name: Adj Close, Length: 252, dtype: float64

In [9]:
data = {
    'option_price': option_price_df.values.ravel(),
    'equity_price': Prices,
    'strike' : [105]*252,
    'rf': [0.05]*252,
    'T' : [(i + 1) / 252 for i in range(len(Prices))],
    'sigma': [0.3]*252
}

df = pd.DataFrame(data, index=stock_price.index)
df

,option_price,equity_price,strike,rf,T,sigma
2022-11-14,0.000000,96.029999,105,0.05,0.003968,0.3
2022-11-15,0.002558,98.720001,105,0.05,0.007937,0.3
2022-11-16,0.038132,98.989998,105,0.05,0.011905,0.3
2022-11-17,0.115306,98.500000,105,0.05,0.015873,0.3
2022-11-18,0.224166,97.800003,105,0.05,0.019841,0.3
...,...,...,...,...,...,...
2023-11-08,22.213422,133.259995,105,0.05,0.984127,0.3
2023-11-09,22.276786,131.690002,105,0.05,0.988095,0.3
2023-11-10,22.344881,134.059998,105,0.05,0.992063,0.3
2023-11-13,22.406966,133.639999,105,0.05,0.996032,0.3


In [10]:
def option_delta(equity_price ,strike , rf, T, sigma):
    d1 = (np.log(equity_price / strike) + (rf + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    N = norm.cdf(d1)
    return N


def option_gamma(equity_price ,strike , rf, T, sigma):
    d1 = (np.log(equity_price / strike) + (rf + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    return norm.pdf(d1)/(equity_price*sigma*np.sqrt(T))


def option_vega(equity_price ,strike , rf, T, sigma):
    d1 = (np.log(equity_price / strike) + (rf + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    vega = equity_price * np.sqrt(T) * norm.pdf(d1)
    return vega


def option_theta(equity_price ,strike , rf, T, sigma):
    d1 = (np.log(equity_price / strike) + (rf + sigma ** 2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    p1 = - equity_price*norm.pdf(d1)*sigma / (2 * np.sqrt(T))
    p2 = rf*strike*np.exp(-rf*T)*norm.cdf(d2) 
    return p1 - p2

In [11]:
def calculate_delta(self):
    return option_delta(self['equity_price'], self['strike'], self['rf'], self['T'],self['sigma'])

def calculate_gamma(self):
    return option_gamma(self['equity_price'], self['strike'], self['rf'], self['T'],self['sigma'])

def calculate_vega(self):
    return option_vega(self['equity_price'], self['strike'], self['rf'], self['T'],self['sigma'])

def calculate_theta(self):
    return option_theta(self['equity_price'], self['strike'], self['rf'], self['T'],self['sigma'])


In [12]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'Date'}, inplace=True)
df

,Date,option_price,equity_price,strike,rf,T,sigma
0,2022-11-14,0.000000,96.029999,105,0.05,0.003968,0.3
1,2022-11-15,0.002558,98.720001,105,0.05,0.007937,0.3
2,2022-11-16,0.038132,98.989998,105,0.05,0.011905,0.3
3,2022-11-17,0.115306,98.500000,105,0.05,0.015873,0.3
4,2022-11-18,0.224166,97.800003,105,0.05,0.019841,0.3
...,...,...,...,...,...,...,...
247,2023-11-08,22.213422,133.259995,105,0.05,0.984127,0.3
248,2023-11-09,22.276786,131.690002,105,0.05,0.988095,0.3
249,2023-11-10,22.344881,134.059998,105,0.05,0.992063,0.3
250,2023-11-13,22.406966,133.639999,105,0.05,0.996032,0.3


In [13]:
df['option_delta'] = df.apply(calculate_delta, axis = 1)
df['option_gamma'] = df.apply(calculate_gamma, axis = 1)
df['option_vega'] =  df.apply(calculate_vega, axis = 1)
df['option_theta'] = df.apply(calculate_theta, axis = 1)
df['APL'] = df['option_price'].diff()
df['HPL'] = np.nan
df['RTPL'] = np.nan
df['VaR'] = np.nan
df

,Date,option_price,equity_price,strike,rf,T,sigma,option_delta,option_gamma,option_vega,option_theta,APL,HPL,RTPL,VaR
0,2022-11-14,0.000000,96.029999,105,0.05,0.003968,0.3,0.000001,0.000003,0.000038,-0.001426,NaN,NaN,NaN,NaN
1,2022-11-15,0.002558,98.720001,105,0.05,0.007937,0.3,0.011322,0.011256,0.261175,-4.991595,0.002558,NaN,NaN,NaN
2,2022-11-16,0.038132,98.989998,105,0.05,0.011905,0.3,0.038686,0.025882,0.905781,-11.601828,0.035574,NaN,NaN,NaN
3,2022-11-17,0.115306,98.500000,105,0.05,0.015873,0.3,0.049386,0.027431,1.267334,-12.215748,0.077173,NaN,NaN,NaN
4,2022-11-18,0.224166,97.800003,105,0.05,0.019841,0.3,0.050877,0.025304,1.440624,-11.135571,0.108861,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2023-11-08,22.213422,133.259995,105,0.05,0.984127,0.3,0.867574,0.005403,28.325261,-8.281400,0.069892,NaN,NaN,NaN
248,2023-11-09,22.276786,131.690002,105,0.05,0.988095,0.3,0.858651,0.005705,29.326371,-8.355285,0.063364,NaN,NaN,NaN
249,2023-11-10,22.344881,134.059998,105,0.05,0.992063,0.3,0.871410,0.005241,28.034250,-8.225221,0.068095,NaN,NaN,NaN
250,2023-11-13,22.406966,133.639999,105,0.05,0.996032,0.3,0.868984,0.005316,28.367032,-8.240580,0.062085,NaN,NaN,NaN


In [14]:
for i in range(1, len(df)):
    df.at[i, 'HPL'] = (df.at[i - 1,'option_delta'])* (df.at[i, 'equity_price'] - df.at[i - 1,'equity_price']) + \
                      (0.5) * (df.at[i - 1, 'option_gamma']) * ((df.at[i, 'equity_price'] - df.at[i - 1, 'equity_price'])** 2) + \
                      df.at[i -1, 'option_theta'] * (df.at[i, 'T'] - df.at[i - 1, 'T'])
df    

,Date,option_price,equity_price,strike,rf,T,sigma,option_delta,option_gamma,option_vega,option_theta,APL,HPL,RTPL,VaR
0,2022-11-14,0.000000,96.029999,105,0.05,0.003968,0.3,0.000001,0.000003,0.000038,-0.001426,NaN,NaN,NaN,NaN
1,2022-11-15,0.002558,98.720001,105,0.05,0.007937,0.3,0.011322,0.011256,0.261175,-4.991595,0.002558,0.000010,NaN,NaN
2,2022-11-16,0.038132,98.989998,105,0.05,0.011905,0.3,0.038686,0.025882,0.905781,-11.601828,0.035574,-0.016341,NaN,NaN
3,2022-11-17,0.115306,98.500000,105,0.05,0.015873,0.3,0.049386,0.027431,1.267334,-12.215748,0.077173,-0.061888,NaN,NaN
4,2022-11-18,0.224166,97.800003,105,0.05,0.019841,0.3,0.050877,0.025304,1.440624,-11.135571,0.108861,-0.076325,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2023-11-08,22.213422,133.259995,105,0.05,0.984127,0.3,0.867574,0.005403,28.325261,-8.281400,0.069892,0.711225,NaN,NaN
248,2023-11-09,22.276786,131.690002,105,0.05,0.988095,0.3,0.858651,0.005705,29.326371,-8.355285,0.063364,-1.388289,NaN,NaN
249,2023-11-10,22.344881,134.059998,105,0.05,0.992063,0.3,0.871410,0.005241,28.034250,-8.225221,0.068095,2.017864,NaN,NaN
250,2023-11-13,22.406966,133.639999,105,0.05,0.996032,0.3,0.868984,0.005316,28.367032,-8.240580,0.062085,-0.398168,NaN,NaN


In [15]:
for i in range(1, len(df)):
    df.at[i,'RTPL'] = (df.at[i - 1,'option_delta'] * df.at[i, 'equity_price']) -(df.at[i - 1, 'option_delta'] * df.at[i - 1, 'equity_price'])
   

df 


,Date,option_price,equity_price,strike,rf,T,sigma,option_delta,option_gamma,option_vega,option_theta,APL,HPL,RTPL,VaR
0,2022-11-14,0.000000,96.029999,105,0.05,0.003968,0.3,0.000001,0.000003,0.000038,-0.001426,NaN,NaN,NaN,NaN
1,2022-11-15,0.002558,98.720001,105,0.05,0.007937,0.3,0.011322,0.011256,0.261175,-4.991595,0.002558,0.000010,0.000003,NaN
2,2022-11-16,0.038132,98.989998,105,0.05,0.011905,0.3,0.038686,0.025882,0.905781,-11.601828,0.035574,-0.016341,0.003057,NaN
3,2022-11-17,0.115306,98.500000,105,0.05,0.015873,0.3,0.049386,0.027431,1.267334,-12.215748,0.077173,-0.061888,-0.018956,NaN
4,2022-11-18,0.224166,97.800003,105,0.05,0.019841,0.3,0.050877,0.025304,1.440624,-11.135571,0.108861,-0.076325,-0.034570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2023-11-08,22.213422,133.259995,105,0.05,0.984127,0.3,0.867574,0.005403,28.325261,-8.281400,0.069892,0.711225,0.742235,NaN
248,2023-11-09,22.276786,131.690002,105,0.05,0.988095,0.3,0.858651,0.005705,29.326371,-8.355285,0.063364,-1.388289,-1.362085,NaN
249,2023-11-10,22.344881,134.059998,105,0.05,0.992063,0.3,0.871410,0.005241,28.034250,-8.225221,0.068095,2.017864,2.034999,NaN
250,2023-11-13,22.406966,133.639999,105,0.05,0.996032,0.3,0.868984,0.005316,28.367032,-8.240580,0.062085,-0.398168,-0.365991,NaN


In [16]:
for i in range(1, len(df)):
    df.at[i, 'VaR'] = df.at[i - 1,'equity_price'] *df.at[i - 1, 'option_delta'] *df.at[i - 1,'sigma'] *np.sqrt(1/250)* norm.ppf(0.01)

df 

,Date,option_price,equity_price,strike,rf,T,sigma,option_delta,option_gamma,option_vega,option_theta,APL,HPL,RTPL,VaR
0,2022-11-14,0.000000,96.029999,105,0.05,0.003968,0.3,0.000001,0.000003,0.000038,-0.001426,NaN,NaN,NaN,NaN
1,2022-11-15,0.002558,98.720001,105,0.05,0.007937,0.3,0.011322,0.011256,0.261175,-4.991595,0.002558,0.000010,0.000003,-0.000005
2,2022-11-16,0.038132,98.989998,105,0.05,0.011905,0.3,0.038686,0.025882,0.905781,-11.601828,0.035574,-0.016341,0.003057,-0.049337
3,2022-11-17,0.115306,98.500000,105,0.05,0.015873,0.3,0.049386,0.027431,1.267334,-12.215748,0.077173,-0.061888,-0.018956,-0.169033
4,2022-11-18,0.224166,97.800003,105,0.05,0.019841,0.3,0.050877,0.025304,1.440624,-11.135571,0.108861,-0.076325,-0.034570,-0.214715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,2023-11-08,22.213422,133.259995,105,0.05,0.984127,0.3,0.867574,0.005403,28.325261,-8.281400,0.069892,0.711225,0.742235,-5.043782
248,2023-11-09,22.276786,131.690002,105,0.05,0.988095,0.3,0.858651,0.005705,29.326371,-8.355285,0.063364,-1.388289,-1.362085,-5.103079
249,2023-11-10,22.344881,134.059998,105,0.05,0.992063,0.3,0.871410,0.005241,28.034250,-8.225221,0.068095,2.017864,2.034999,-4.991090
250,2023-11-13,22.406966,133.639999,105,0.05,0.996032,0.3,0.868984,0.005316,28.367032,-8.240580,0.062085,-0.398168,-0.365991,-5.156414


In [18]:
#Baktesting
required_columns = ['APL', 'HPL', 'RTPL','VaR']
backtesting = df[required_columns].iloc[1:].copy()

backtesting['APL_exceptions'] = np.where(backtesting['VaR'] > backtesting['APL'],1,0)
backtesting['HPL_exceptions'] = np.where(backtesting['VaR'] > backtesting['HPL'],1,0)
backtesting['APL_exceptions'].sum()
backtesting['HPL_exceptions'].sum()
backtesting_exception = max(backtesting['APL_exceptions'].sum(),backtesting['HPL_exceptions'].sum())
backtesting_exception

2

In [19]:
#PLAT, spearman rank correlation
from scipy.stats import spearmanr
correlation = spearmanr(backtesting['HPL'], backtesting['RTPL'])
correlation

SpearmanrResult(correlation=0.9986734964902294, pvalue=8e-323)

In [20]:
#PLAT, KS - Test
plat_df = pd.DataFrame({
    'HPL_sorted': backtesting['HPL'].sort_values(ascending=True).values,
    'RTPL_sorted': backtesting['RTPL'].sort_values(ascending=True).values
})
plat_df.reset_index(drop=True, inplace=True)  

hplsd = backtesting['HPL'].std()
rtplsd = backtesting['RTPL'].std()
plat_df['HPL_CDF'] = norm.cdf(plat_df['HPL_sorted'], backtesting['HPL'].mean(), hplsd)
plat_df['RTPL_CDF'] = norm.cdf(plat_df['RTPL_sorted'], backtesting['RTPL'].mean(),rtplsd)
plat_df

,HPL_sorted,RTPL_sorted,HPL_CDF,RTPL_CDF
0,-11.796134,-12.145409,1.853405e-14,3.867238e-15
1,-4.403183,-5.120804,2.155756e-03,4.759528e-04
2,-4.261920,-4.315758,2.852241e-03,2.616108e-03
3,-3.825759,-3.833516,6.458590e-03,6.465141e-03
4,-3.202994,-3.233236,1.836554e-02,1.769274e-02
...,...,...,...,...
246,3.233249,3.227052,9.781718e-01,9.775011e-01
247,3.372292,3.261315,9.824013e-01,9.786414e-01
248,4.089217,3.384161,9.948182e-01,9.823366e-01
249,4.193764,4.097311,9.957327e-01,9.947383e-01


In [21]:
KS_test = max( abs( plat_df['HPL_CDF'] - plat_df['RTPL_CDF'] ) )
print('Kolmogorov-Smirnov test metric is', KS_test)

Kolmogorov-Smirnov test metric is 0.022623059643418097


## IF CORRELATION IS > 0.8 AND KS - STATISTIC IS < 0.09 THEN TRADING DESK IS ALLOWED FOR IMA